# Initialization

In [1]:
import os
import numpy as np
import pickle
import json
import pandas as pd
from grabbit import Layout
from mne import read_epochs, grand_average, write_evokeds, read_evokeds
from mne import pick_types
from mne.viz import plot_compare_evokeds
from msit import CH_NAMES
import matplotlib.pyplot as plt

CH_NAMES = CH_NAMES[:-10] + ['HEOG', 'VEOG'] + CH_NAMES[-10:]
# load subject names to process
pipeline_root = '../data/derivatives/eeg_sensor_erps'
layout = Layout('../data', '../data/grabbit_config.json')
subjects = np.array(sorted(layout.get(target='subject', 
                                      modality='eeg',
                                      return_type='id')))

# load eeg configurations
with open('../config.json') as f:    
    config = json.load(f)

# load behavior
behavior = pd.read_csv('../data/derivatives/behavior/group_data.tsv', 
                       na_values='n/a', sep='\t') 
behavior = behavior[behavior.modality == 'eeg']

# Make directory structure
pipeline_root = '../data/derivatives/eeg_sensor_erps'
if not os.path.exists(pipeline_root):
    os.makedirs(pipeline_root)

layout = Layout('../data', '../data/grabbit_config.json')
subjects = layout.get(target='subject', modality='eeg', return_type='id')

for subject in subjects + ['group']:
    if not os.path.exists('%s/%s/erps' % (pipeline_root, subject)):
        os.makedirs('%s/%s/erps' % (pipeline_root, subject))

# Make Evoked Data

In [4]:
for epo_type, epo_times in zip(config['epoch_types'], 
                               config['epoch_times']):
    
    evokeds = {'incongruent': [], 'congruent': []}
    
    for subject in subjects:
        if subject == 'sub-hc013':
            continue
        print(subject)

        sub_behavior = behavior[behavior.participant_id == subject]
        sub_behavior = sub_behavior[sub_behavior.no_response != 1].reset_index()
    
        # load, crop, and interpolate cleaned epochs
        epo_file = layout.get(subject=subject, 
                              derivative='eeg_preprocessing', 
                              extensions='%s_cleaned-epo.fif' % epo_type)[0].filename
        epochs = read_epochs(epo_file, verbose=False)
        epochs.crop(epo_times[0], epo_times[1])
        epochs.interpolate_bads(reset_bads=True)

        # drop bad eeg trials from behavior
        ar_file = layout.get(subject=subject, 
                             derivative='eeg_preprocessing', 
                             extensions='%s_ar.pkl' % epo_type)[0].filename
        ar = pickle.load(open(ar_file, 'r'))
        sub_behavior = sub_behavior.drop(ar.bad_epochs_idx).reset_index()
        
        # drop bad behavior trials from eeg and behavior
        behavior_exclude = np.where(np.sum(sub_behavior[['fast_rt', 'error', 
                                                         'post_error']], 
                                           axis=1))[0]
        sub_behavior = sub_behavior.drop(behavior_exclude)
        epochs.drop(behavior_exclude)
        
        # inject event labels
        events = sub_behavior.trial_type.astype('category').cat.codes
        if subject == 'sub-pp012':
            events = events[1:]
        epochs.event_id = {'incongruent': 1, 'congruent': 0}
        epochs.events[:, -1] = events
        
        # write evokeds to file
        picks = pick_types(epochs.info, eeg=True, eog=True, meg=False)
        evos = [epochs['incongruent'].average(picks), 
                epochs['congruent'].average(picks)]
        print(evos[0].ch_names)
        write_evokeds('%s/%s/%s_%s-ave.fif' % (pipeline_root,
                                               subject, subject,
                                               epo_type), evos)
        # accumulate evokeds for group average
        evokeds['incongruent'].append(evos[0])
        evokeds['congruent'].append(evos[1])
    
    # calculate group average and save
    evos = [grand_average(evokeds['incongruent']),
            grand_average(evokeds['congruent'])]
    write_evokeds('%s/group/group_%s-ave.fif' % (pipeline_root,
                                                 epo_type), evos)


print('Done!')

sub-hc001
Computing interpolation matrix from 60 sensor positions
Interpolating 10 sensors
[u'Fp1', u'Fpz', u'Fp2', u'AF7', u'AF3', u'AFz', u'AF4', u'AF8', u'F7', u'F5', u'F3', u'F1', u'Fz', u'F2', u'F4', u'F6', u'F8', u'FT9', u'FT7', u'FC5', u'FC3', u'FC1', u'FCz', u'FC2', u'FC4', u'FC6', u'FT8', u'FT10', u'T9', u'T7', u'C5', u'C3', u'C1', u'Cz', u'C2', u'C4', u'C6', u'T8', u'T10', u'TP9', u'TP7', u'CP5', u'CP3', u'CP1', u'CPz', u'CP2', u'CP4', u'CP6', u'TP8', u'TP10', u'P9', u'P7', u'P5', u'P3', u'P1', u'Pz', u'P2', u'P4', u'P6', u'P8', u'HEOG', u'VEOG', u'P10', u'PO7', u'PO3', u'P0z', u'PO4', u'PO8', u'O1', u'Oz', u'O2', u'Iz']
sub-hc002
Computing interpolation matrix from 61 sensor positions
Interpolating 9 sensors
[u'Fp1', u'Fpz', u'Fp2', u'AF7', u'AF3', u'AFz', u'AF4', u'AF8', u'F7', u'F5', u'F3', u'F1', u'Fz', u'F2', u'F4', u'F6', u'F8', u'FT9', u'FT7', u'FC5', u'FC3', u'FC1', u'FCz', u'FC2', u'FC4', u'FC6', u'FT8', u'FT10', u'T9', u'T7', u'C5', u'C3', u'C1', u'Cz', u'C2', u'C4'

Computing interpolation matrix from 62 sensor positions
Interpolating 8 sensors
[u'Fp1', u'Fpz', u'Fp2', u'AF7', u'AF3', u'AFz', u'AF4', u'AF8', u'F7', u'F5', u'F3', u'F1', u'Fz', u'F2', u'F4', u'F6', u'F8', u'FT9', u'FT7', u'FC5', u'FC3', u'FC1', u'FCz', u'FC2', u'FC4', u'FC6', u'FT8', u'FT10', u'T9', u'T7', u'C5', u'C3', u'C1', u'Cz', u'C2', u'C4', u'C6', u'T8', u'T10', u'TP9', u'TP7', u'CP5', u'CP3', u'CP1', u'CPz', u'CP2', u'CP4', u'CP6', u'TP8', u'TP10', u'P9', u'P7', u'P5', u'P3', u'P1', u'Pz', u'P2', u'P4', u'P6', u'P8', u'HEOG', u'VEOG', u'P10', u'PO7', u'PO3', u'P0z', u'PO4', u'PO8', u'O1', u'Oz', u'O2', u'Iz']
sub-hc016
Computing interpolation matrix from 68 sensor positions
Interpolating 2 sensors
[u'Fp1', u'Fpz', u'Fp2', u'AF7', u'AF3', u'AFz', u'AF4', u'AF8', u'F7', u'F5', u'F3', u'F1', u'Fz', u'F2', u'F4', u'F6', u'F8', u'FT9', u'FT7', u'FC5', u'FC3', u'FC1', u'FCz', u'FC2', u'FC4', u'FC6', u'FT8', u'FT10', u'T9', u'T7', u'C5', u'C3', u'C1', u'Cz', u'C2', u'C4', u'C6', u'

Computing interpolation matrix from 60 sensor positions
Interpolating 10 sensors
[u'Fp1', u'Fpz', u'Fp2', u'AF7', u'AF3', u'AFz', u'AF4', u'AF8', u'F7', u'F5', u'F3', u'F1', u'Fz', u'F2', u'F4', u'F6', u'F8', u'FT9', u'FT7', u'FC5', u'FC3', u'FC1', u'FCz', u'FC2', u'FC4', u'FC6', u'FT8', u'FT10', u'T9', u'T7', u'C5', u'C3', u'C1', u'Cz', u'C2', u'C4', u'C6', u'T8', u'T10', u'TP9', u'TP7', u'CP5', u'CP3', u'CP1', u'CPz', u'CP2', u'CP4', u'CP6', u'TP8', u'TP10', u'P9', u'P7', u'P5', u'P3', u'P1', u'Pz', u'P2', u'P4', u'P6', u'P8', u'HEOG', u'VEOG', u'P10', u'PO7', u'PO3', u'P0z', u'PO4', u'PO8', u'O1', u'Oz', u'O2', u'Iz']
sub-hc029
Computing interpolation matrix from 50 sensor positions
Interpolating 20 sensors
[u'Fp1', u'Fpz', u'Fp2', u'AF7', u'AF3', u'AFz', u'AF4', u'AF8', u'F7', u'F5', u'F3', u'F1', u'Fz', u'F2', u'F4', u'F6', u'F8', u'FT9', u'FT7', u'FC5', u'FC3', u'FC1', u'FCz', u'FC2', u'FC4', u'FC6', u'FT8', u'FT10', u'T9', u'T7', u'C5', u'C3', u'C1', u'Cz', u'C2', u'C4', u'C6', 

Computing interpolation matrix from 61 sensor positions
Interpolating 9 sensors
[u'Fp1', u'Fpz', u'Fp2', u'AF7', u'AF3', u'AFz', u'AF4', u'AF8', u'F7', u'F5', u'F3', u'F1', u'Fz', u'F2', u'F4', u'F6', u'F8', u'FT9', u'FT7', u'FC5', u'FC3', u'FC1', u'FCz', u'FC2', u'FC4', u'FC6', u'FT8', u'FT10', u'T9', u'T7', u'C5', u'C3', u'C1', u'Cz', u'C2', u'C4', u'C6', u'T8', u'T10', u'TP9', u'TP7', u'CP5', u'CP3', u'CP1', u'CPz', u'CP2', u'CP4', u'CP6', u'TP8', u'TP10', u'P9', u'P7', u'P5', u'P3', u'P1', u'Pz', u'P2', u'P4', u'P6', u'P8', u'HEOG', u'VEOG', u'P10', u'PO7', u'PO3', u'P0z', u'PO4', u'PO8', u'O1', u'Oz', u'O2', u'Iz']
sub-pp002
Computing interpolation matrix from 64 sensor positions
Interpolating 6 sensors
[u'Fp1', u'Fpz', u'Fp2', u'AF7', u'AF3', u'AFz', u'AF4', u'AF8', u'F7', u'F5', u'F3', u'F1', u'Fz', u'F2', u'F4', u'F6', u'F8', u'FT9', u'FT7', u'FC5', u'FC3', u'FC1', u'FCz', u'FC2', u'FC4', u'FC6', u'FT8', u'FT10', u'T9', u'T7', u'C5', u'C3', u'C1', u'Cz', u'C2', u'C4', u'C6', u'

Computing interpolation matrix from 60 sensor positions
Interpolating 10 sensors
[u'Fp1', u'Fpz', u'Fp2', u'AF7', u'AF3', u'AFz', u'AF4', u'AF8', u'F7', u'F5', u'F3', u'F1', u'Fz', u'F2', u'F4', u'F6', u'F8', u'FT9', u'FT7', u'FC5', u'FC3', u'FC1', u'FCz', u'FC2', u'FC4', u'FC6', u'FT8', u'FT10', u'T9', u'T7', u'C5', u'C3', u'C1', u'Cz', u'C2', u'C4', u'C6', u'T8', u'T10', u'TP9', u'TP7', u'CP5', u'CP3', u'CP1', u'CPz', u'CP2', u'CP4', u'CP6', u'TP8', u'TP10', u'P9', u'P7', u'P5', u'P3', u'P1', u'Pz', u'P2', u'P4', u'P6', u'P8', u'HEOG', u'VEOG', u'P10', u'PO7', u'PO3', u'P0z', u'PO4', u'PO8', u'O1', u'Oz', u'O2', u'Iz']
sub-pp015
Computing interpolation matrix from 67 sensor positions
Interpolating 3 sensors
[u'Fp1', u'Fpz', u'Fp2', u'AF7', u'AF3', u'AFz', u'AF4', u'AF8', u'F7', u'F5', u'F3', u'F1', u'Fz', u'F2', u'F4', u'F6', u'F8', u'FT9', u'FT7', u'FC5', u'FC3', u'FC1', u'FCz', u'FC2', u'FC4', u'FC6', u'FT8', u'FT10', u'T9', u'T7', u'C5', u'C3', u'C1', u'Cz', u'C2', u'C4', u'C6', u

Computing interpolation matrix from 66 sensor positions
Interpolating 4 sensors
[u'Fp1', u'Fpz', u'Fp2', u'AF7', u'AF3', u'AFz', u'AF4', u'AF8', u'F7', u'F5', u'F3', u'F1', u'Fz', u'F2', u'F4', u'F6', u'F8', u'FT9', u'FT7', u'FC5', u'FC3', u'FC1', u'FCz', u'FC2', u'FC4', u'FC6', u'FT8', u'FT10', u'T9', u'T7', u'C5', u'C3', u'C1', u'Cz', u'C2', u'C4', u'C6', u'T8', u'T10', u'TP9', u'TP7', u'CP5', u'CP3', u'CP1', u'CPz', u'CP2', u'CP4', u'CP6', u'TP8', u'TP10', u'P9', u'P7', u'P5', u'P3', u'P1', u'Pz', u'P2', u'P4', u'P6', u'P8', u'HEOG', u'VEOG', u'P10', u'PO7', u'PO3', u'P0z', u'PO4', u'PO8', u'O1', u'Oz', u'O2', u'Iz']
sub-hc012
Computing interpolation matrix from 64 sensor positions
Interpolating 6 sensors
[u'Fp1', u'Fpz', u'Fp2', u'AF7', u'AF3', u'AFz', u'AF4', u'AF8', u'F7', u'F5', u'F3', u'F1', u'Fz', u'F2', u'F4', u'F6', u'F8', u'FT9', u'FT7', u'FC5', u'FC3', u'FC1', u'FCz', u'FC2', u'FC4', u'FC6', u'FT8', u'FT10', u'T9', u'T7', u'C5', u'C3', u'C1', u'Cz', u'C2', u'C4', u'C6', u'

Computing interpolation matrix from 62 sensor positions
Interpolating 8 sensors
[u'Fp1', u'Fpz', u'Fp2', u'AF7', u'AF3', u'AFz', u'AF4', u'AF8', u'F7', u'F5', u'F3', u'F1', u'Fz', u'F2', u'F4', u'F6', u'F8', u'FT9', u'FT7', u'FC5', u'FC3', u'FC1', u'FCz', u'FC2', u'FC4', u'FC6', u'FT8', u'FT10', u'T9', u'T7', u'C5', u'C3', u'C1', u'Cz', u'C2', u'C4', u'C6', u'T8', u'T10', u'TP9', u'TP7', u'CP5', u'CP3', u'CP1', u'CPz', u'CP2', u'CP4', u'CP6', u'TP8', u'TP10', u'P9', u'P7', u'P5', u'P3', u'P1', u'Pz', u'P2', u'P4', u'P6', u'P8', u'HEOG', u'VEOG', u'P10', u'PO7', u'PO3', u'P0z', u'PO4', u'PO8', u'O1', u'Oz', u'O2', u'Iz']
sub-hc026
Computing interpolation matrix from 66 sensor positions
Interpolating 4 sensors
[u'Fp1', u'Fpz', u'Fp2', u'AF7', u'AF3', u'AFz', u'AF4', u'AF8', u'F7', u'F5', u'F3', u'F1', u'Fz', u'F2', u'F4', u'F6', u'F8', u'FT9', u'FT7', u'FC5', u'FC3', u'FC1', u'FCz', u'FC2', u'FC4', u'FC6', u'FT8', u'FT10', u'T9', u'T7', u'C5', u'C3', u'C1', u'Cz', u'C2', u'C4', u'C6', u'

Computing interpolation matrix from 62 sensor positions
Interpolating 8 sensors
[u'Fp1', u'Fpz', u'Fp2', u'AF7', u'AF3', u'AFz', u'AF4', u'AF8', u'F7', u'F5', u'F3', u'F1', u'Fz', u'F2', u'F4', u'F6', u'F8', u'FT9', u'FT7', u'FC5', u'FC3', u'FC1', u'FCz', u'FC2', u'FC4', u'FC6', u'FT8', u'FT10', u'T9', u'T7', u'C5', u'C3', u'C1', u'Cz', u'C2', u'C4', u'C6', u'T8', u'T10', u'TP9', u'TP7', u'CP5', u'CP3', u'CP1', u'CPz', u'CP2', u'CP4', u'CP6', u'TP8', u'TP10', u'P9', u'P7', u'P5', u'P3', u'P1', u'Pz', u'P2', u'P4', u'P6', u'P8', u'HEOG', u'VEOG', u'P10', u'PO7', u'PO3', u'P0z', u'PO4', u'PO8', u'O1', u'Oz', u'O2', u'Iz']
sub-hc044
Computing interpolation matrix from 62 sensor positions
Interpolating 8 sensors
[u'Fp1', u'Fpz', u'Fp2', u'AF7', u'AF3', u'AFz', u'AF4', u'AF8', u'F7', u'F5', u'F3', u'F1', u'Fz', u'F2', u'F4', u'F6', u'F8', u'FT9', u'FT7', u'FC5', u'FC3', u'FC1', u'FCz', u'FC2', u'FC4', u'FC6', u'FT8', u'FT10', u'T9', u'T7', u'C5', u'C3', u'C1', u'Cz', u'C2', u'C4', u'C6', u'

Computing interpolation matrix from 66 sensor positions
Interpolating 4 sensors
[u'Fp1', u'Fpz', u'Fp2', u'AF7', u'AF3', u'AFz', u'AF4', u'AF8', u'F7', u'F5', u'F3', u'F1', u'Fz', u'F2', u'F4', u'F6', u'F8', u'FT9', u'FT7', u'FC5', u'FC3', u'FC1', u'FCz', u'FC2', u'FC4', u'FC6', u'FT8', u'FT10', u'T9', u'T7', u'C5', u'C3', u'C1', u'Cz', u'C2', u'C4', u'C6', u'T8', u'T10', u'TP9', u'TP7', u'CP5', u'CP3', u'CP1', u'CPz', u'CP2', u'CP4', u'CP6', u'TP8', u'TP10', u'P9', u'P7', u'P5', u'P3', u'P1', u'Pz', u'P2', u'P4', u'P6', u'P8', u'HEOG', u'VEOG', u'P10', u'PO7', u'PO3', u'P0z', u'PO4', u'PO8', u'O1', u'Oz', u'O2', u'Iz']
sub-pp012
Computing interpolation matrix from 60 sensor positions
Interpolating 10 sensors
[u'Fp1', u'Fpz', u'Fp2', u'AF7', u'AF3', u'AFz', u'AF4', u'AF8', u'F7', u'F5', u'F3', u'F1', u'Fz', u'F2', u'F4', u'F6', u'F8', u'FT9', u'FT7', u'FC5', u'FC3', u'FC1', u'FCz', u'FC2', u'FC4', u'FC6', u'FT8', u'FT10', u'T9', u'T7', u'C5', u'C3', u'C1', u'Cz', u'C2', u'C4', u'C6', u

# Plot Congruent vs. Incongruent ERPs

In [11]:
from ipywidgets import interact

def plot_erp(subject, ch):

    fig, axs = plt.subplots(1, 2, figsize=(24, 8))

    for i, epo_type in enumerate(config['epoch_types']):

        ax = axs[i]

        evokeds = read_evokeds('%s/%s/%s_%s-ave.fif' % (pipeline_root,
                                                        subject,
                                                        subject,
                                                        epo_type),
                              verbose=False)
        print(evokeds[0].ch_names)
        pick = evokeds[0].ch_names.index(ch)
        evokeds = dict((config['conditions'][i], evokeds[i]) 
                       for i in range(len(evokeds)))
        plot_compare_evokeds(evokeds, picks=pick, axes=ax, show=False)
        ax.set_title('%s-locked' % epo_type)
#         ax.set_ylim((-200, 200))
        ax.set_ylim((-6, 6))
        ax.set_xticks(np.arange(-.5, 1.75, .25))
        ax.set_xticklabels(np.arange(-.5, 1.75, .25))
        ax.axvline(0, linestyle='--', color='k')

    plt.suptitle('%s %s ERP' % (subject, ch))
    plt.show()

interact(plot_erp, subject=subjects + ['group'], ch=CH_NAMES)

Widget Javascript not detected.  It may not be installed or enabled properly.


<function __main__.plot_erp>

In [7]:
plot_compare_evokeds?